In [1]:
import torch, torchvision
import sys # Python system library needed to load custom functions
import math # module with access to mathematical functions
import os # for changing the directory

import numpy as np  # for performing calculations on numerical arrays
import pandas as pd  # home of the DataFrame construct, _the_ most important object for Data Science
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt  # allows creation of insightful plots

sys.path.append('../../audio_preprocessing')
sys.path.append('../../src')
sys.path.append('../../model_training_utils')


from normalised_image_dataset import NormalisedImageDataSet
from gdsc_utils import PROJECT_DIR
import model_training
import model_eval

os.chdir(PROJECT_DIR) # changing our directory to root

In [2]:
df = pd.read_csv('data/normalised_stft_image_train_val_with_upsample.csv')
df.head()

,Unnamed: 0,file_path,label
0,0,data/image_train_val_with_upsample_stft/0.pt,56
1,1,data/image_train_val_with_upsample_stft/1.pt,56
2,2,data/image_train_val_with_upsample_stft/2.pt,56
3,3,data/image_train_val_with_upsample_stft/3.pt,56
4,4,data/image_train_val_with_upsample_stft/4.pt,56


In [3]:
train_df_list = []
val_df_list = []

for i in range(66):
    my_df = df[df["label"] == i]
    current_train_df, current_val_df = train_test_split(my_df, test_size=0.2)
    train_df_list.append(current_train_df)
    val_df_list.append(current_val_df)

df_train = pd.concat(train_df_list, ignore_index=True)
df_val = pd.concat(val_df_list, ignore_index=True)

In [4]:
df_train.to_csv("model_definition_and_training/resnext50/resnext50_train_stft.csv", index=False)
df_val.to_csv("model_definition_and_training/resnext50/resnext50_val_stft.csv", index=False)

In [5]:
train_dataset = NormalisedImageDataSet(
    df=df_train, 
    shuffle=True
)

val_dataset = NormalisedImageDataSet(
    df=df_val, 
    shuffle=False
)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=28)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=28)

In [6]:
device = model_training.get_device()

In [7]:
from torchvision.models import resnext50_32x4d
import torch.nn as nn
import torch.optim as optim

#resnet_model = resnet34(weights=ResNet34_Weights.DEFAULT)
resnext_model = resnext50_32x4d(num_classes=66)
resnext_model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnext_model = resnext_model.to(device)

In [8]:
print(resnext_model)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [9]:
optimizer = optim.Adam(resnext_model.parameters(), amsgrad=True)
loss = nn.CrossEntropyLoss()

In [22]:
model_training.training(
    model=resnext_model, 
    optimizer=optimizer, 
    loss_fn=loss, 
    train_dataloader=train_dataloader, 
    val_dataloader=val_dataloader, 
    model_path="models/rexnext50_train_and_val_stft", 
    start_epoch=8,
    early_stop_thresh=50,
)

KeyboardInterrupt: 

In [23]:
torch.save(resnext_model, 'models/rexnext50_train_and_val_stft/resnext_model_epoch_8.pth')